In [4]:
import csv
import sys
import re
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from datetime import datetime
import numpy as np

In [6]:
# Import the list of winners and nominees  
oscars_lst = pd.read_csv("/Users/hazelchui/Desktop/UChicago/Perspectives/final_oscars_list.csv")
oscars_lst = oscars_lst.fillna(0)
oscars_lst['release_year'] = (oscars_lst['year']-1).astype(str)
oscars_lst.head()

,year,movie,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,best_doc_short,best_doc_short_won,best_live_short,best_live_short_won,best_animated_short,best_animated_short_won,release_year
0,2015,Birdman or (The Unexpected Virtue of Ignorance...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
1,2015,"American Sniper – Clint Eastwood, Robert Loren...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
2,2015,Boyhood – Richard Linklater and Cathleen Suthe...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
3,2015,"The Grand Budapest Hotel – Wes Anderson, Scott...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
4,2015,"The Imitation Game – Nora Grossman, Ido Ostrow...",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014


In [7]:
# Clean movie name
oscars_lst['movie'] = oscars_lst['movie'].str.extract(r'(.*?)\xa0–')
oscars_lst
oscars_lst.rename(columns={'movie':'movie_name'}, inplace = True)

In [8]:
# 474 movies
oscars_lst = oscars_lst.dropna(subset=['movie_name'])
oscars_lst.head()

,year,movie_name,best_picture,best_picture_won,best_animated_feature,best_animated_feature_won,best_foreign,best_foreign_won,best_doc,best_doc_won,best_doc_short,best_doc_short_won,best_live_short,best_live_short_won,best_animated_short,best_animated_short_won,release_year
0,2015,Birdman or (The Unexpected Virtue of Ignorance),1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
1,2015,American Sniper,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
2,2015,Boyhood,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
3,2015,The Grand Budapest Hotel,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014
4,2015,The Imitation Game,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014


In [14]:
# Concatenate release year and movie name
release_year = list(oscars_lst['release_year'])
movie = list(oscars_lst['movie_name'])
movie_lst = list(zip(release_year, movie))

In [28]:
# Grab the imdb link for each movie
def get_imdb_links(movie_lst):     
    
    driver = webdriver.Safari()
    driver.get("https://www.google.com/")
    m = driver.find_element(By.NAME, "q")
    link_lst = []
    for idx, (release_year, movie_name) in enumerate(movie_lst):
        time.sleep(1)
        driver.get("https://www.google.com/")
        m = driver.find_element(By.NAME, "q")
        m.send_keys(movie_name + " IMDB " + release_year)
        m.send_keys(Keys.ENTER)
        time.sleep(2)
        elems = driver.find_elements(By.CSS_SELECTOR,".yuRUbf [href]")
        link = [elem.get_attribute('href') for elem in elems][0]
        if link.startswith('https://www.imdb.com/title/'):
            link_lst.append((release_year, movie_name, link))
            print(idx, movie_name)
        else:
            print('Not on IMDB')
    driver.close()
    return link_lst

In [29]:
results = get_imdb_links([i for i in movie_lst[0:5]])

0 Birdman or (The Unexpected Virtue of Ignorance)
1 American Sniper
2 Boyhood
3 The Grand Budapest Hotel
4 The Imitation Game


In [25]:
# Display the first 5 results
results

[('2014',
  'Birdman or (The Unexpected Virtue of Ignorance)',
  'https://www.imdb.com/title/tt2562232/'),
 ('2014', 'American Sniper', 'https://www.imdb.com/title/tt2179136/'),
 ('2014', 'Boyhood', 'https://www.imdb.com/title/tt1065073/'),
 ('2014', 'The Grand Budapest Hotel', 'https://www.imdb.com/title/tt2278388/'),
 ('2014', 'The Imitation Game', 'https://www.imdb.com/title/tt2084970/')]

In [30]:
# Scrape other information (box office, opening date, no. of other oscars, etc)
def get_other_info(results):
    
    csv_file = open('info.csv', 'a', encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['release_year', 'movie_name', 'link', 
                         'oscar_num', 'opening_box', 'opening_date']) 
    driver = webdriver.Safari()
    for idx, (release_year, movie_name, link) in enumerate(results):
        driver.get(link)
        time.sleep(3)
        
        # No. of other oscars 
        oscar_num = re.findall(r'Won (.*?) Oscars',  driver.find_element(By.CSS_SELECTOR, "#__next>main >div>section>div>section>div>div>section>div>ul>li>a").text)
        if oscar_num != []:
            oscar_num = int(oscar_num[0])
        else:
            oscar_num = 0

        # Opening Box office & opening date 
        opening_weekend = driver.find_elements(By.CSS_SELECTOR, "#__next > main > div > section > div > section > div > div > section > div > ul > li > div > ul > li > span")
        op = []
        for i in opening_weekend:
            op.append(i.text)
        for index, mo in enumerate(op):
            opening_date = re.findall(r'[A-Z][a-z][a-z]\s[0-9]+[,]\s[0-9]+', mo)
            if opening_date != []:
                opening_d = opening_date[0] 
                opening_box = op[index-1]
                csv_writer.writerow([release_year, movie_name, link, 
                         oscar_num, opening_box, opening_d])
                break
            elif index == len(op)-1:
                opening_d = 'NA'
                opening_box = 'NA'
                csv_writer.writerow([release_year, movie_name, link, 
                 oscar_num, opening_box, opening_d])
        print(idx, movie_name)
        
    driver.close()                                    
    

In [32]:
info = get_other_info(results)

0 Birdman or (The Unexpected Virtue of Ignorance)
1 American Sniper
2 Boyhood
3 The Grand Budapest Hotel
4 The Imitation Game


In [36]:
# Display the first 5 results
info = pd.read_csv('info.csv')
info.head()

,release_year,movie_name,link,oscar_num,opening_box,opening_date
0,2014,Birdman or (The Unexpected Virtue of Ignorance),https://www.imdb.com/title/tt2562232/,4,"$424,397","Oct 19, 2014"
1,2014,American Sniper,https://www.imdb.com/title/tt2179136/,0,"$633,456","Dec 28, 2014"
2,2014,Boyhood,https://www.imdb.com/title/tt1065073/,0,"$387,618","Jul 13, 2014"
3,2014,The Grand Budapest Hotel,https://www.imdb.com/title/tt2278388/,4,"$811,166","Mar 9, 2014"
4,2014,The Imitation Game,https://www.imdb.com/title/tt2084970/,0,"$479,352","Nov 30, 2014"


In [37]:
# Scrape all the reviews from each movie
def get_reviews(links):
    csv_file = open('reviews_0729.csv', 'a', encoding="utf-8")
    csv_writer = csv.writer(csv_file)
    csv_writer.writerow(['release_year', 'movie_id','movie_name', 'date', 'user_name', 
                         'review', 'rating']) 
    for idx, (release_year, movie_name, link) in enumerate(links): 
        start_url = link + 'reviews?ref_=tt_urv'
        link = link + 'reviews/_ajax'

        params = {
            'ref_': 'undefined',
            'paginationKey': ''
        }

        with requests.Session() as s:
            s.headers['User-Agent'] = 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
            res = s.get(start_url)
            while True:
                soup = BeautifulSoup(res.text,"lxml")
                for item in soup.select(".review-container"):
                    try:
                        rating_fraction = item.select_one("span.rating-other-user-rating").get_text(strip=True)
                        rating = re.findall(r'(.*?)/10', rating_fraction)[0]
                    except:
                        rating = 'NA'
                    movie_id = re.findall(r'https://www.imdb.com/title/(.*?)/', link)[0]
                    reviewer_name = item.select_one("span.display-name-link > a").get_text(strip=True)
                    date = item.select_one("span.review-date").get_text(strip=True)
                    review = item.select_one(".content > .text").get_text(strip=True)
                    csv_writer.writerow([release_year, movie_id, movie_name, date, reviewer_name,
                             review, rating])
                
                try:
                    pagination_key = soup.select_one(".load-more-data[data-key]").get("data-key")
                except AttributeError:
                    break
                params['paginationKey'] = pagination_key
                res = s.get(link, params=params)
        print(idx, movie_name)

In [38]:
get_reviews(results)
reviews = pd.read_csv('reviews_0729.csv')
reviews.head()

0 Birdman or (The Unexpected Virtue of Ignorance)
1 American Sniper
2 Boyhood
3 The Grand Budapest Hotel
4 The Imitation Game


,release_year,movie_id,movie_name,date,user_name,review,rating
0,2014,tt2562232,Birdman or (The Unexpected Virtue of Ignorance),19 September 2018,PIST-OFF,With all due respect to Eastwood's American Sn...,10.0
1,2014,tt2562232,Birdman or (The Unexpected Virtue of Ignorance),9 June 2017,TheLittleSongbird,Despite the near-universal acclaim from critic...,9.0
2,2014,tt2562232,Birdman or (The Unexpected Virtue of Ignorance),1 January 2015,CalRhys,"Whilst viewing 'Birdman', I spent the first ho...",NaN
3,2014,tt2562232,Birdman or (The Unexpected Virtue of Ignorance),24 February 2015,apbryant,I have to say I am shocked and how many bad re...,10.0
4,2014,tt2562232,Birdman or (The Unexpected Virtue of Ignorance),27 December 2016,theocharous_an,I have a tendency to check user reviews on IMD...,9.0
